In [1]:
import pyspark
import pandas as pd
import h2o
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator

Associated Tutorial: https://github.com/h2oai/h2o-tutorials/blob/master/tutorials/glrm/glrm.census.labor.violations.ipynb

In [2]:
print("PySpark Version:{}".format(pyspark.__version__))
print("H2O Version:{}".format(h2o.__version__))

PySpark Version:2.4.3
H2O Version:3.26.0.2


In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_171"; Java(TM) SE Runtime Environment (build 1.8.0_171-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /Users/RobertSmith/PycharmProjects/experimentation/venv/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/v4/tx83b2212tb8_g8bcxd74wrc0000gn/T/tmplee45dy1
  JVM stdout: /var/folders/v4/tx83b2212tb8_g8bcxd74wrc0000gn/T/tmplee45dy1/h2o_RobertSmith_started_from_python.out
  JVM stderr: /var/folders/v4/tx83b2212tb8_g8bcxd74wrc0000gn/T/tmplee45dy1/h2o_RobertSmith_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,"1 year, 7 months and 5 days !!!"
H2O cluster name:,H2O_from_python_RobertSmith_thq0jz
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [4]:
# Import and parse ACS 2013 5-year DP02 demographic data
acs_orig = h2o.import_file("ACS_13_5YR_DP02_cleaned.csv", col_types = (["enum"] + ["numeric"]*149))
acs_orig.describe()

acs_zcta_col = acs_orig["ZCTA5"].asfactor()
acs_full = acs_orig.drop("ZCTA5")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:32989
Cols:150




,ZCTA5,total_households,family_households,families_w_children_under_18,married_families,married_w_children_under_18,male_no_wife,male_no_wife_w_children_under_18,female_no_husband,female_no_husband_w_children_under_18,nonfamily_households,living_alone,living_alone_65_and_over,households_w_people_under_18,households_w_people_over_65,average_household_size,average_family_size,pop_in_households,pop_householder,pop_spouse,pop_child,pop_other_relatives,pop_nonrelatives,pop_unmarried_partner,males_15_and_over,males_15_and_over_unmarried,males_15_and_over_married,males_15_and_over_separated,males_15_and_over_widowed,males_15_and_over_divorced,females_15_and_over,females_15_and_over_unmarried,females_15_and_over_married,females_15_and_over_separated,females_15_and_over_widowed,females_15_and_over_divorced,women_birth_past_year,unmarried_women_birth_past_year,unmarried_women_birth_past_year_per_1000,women_birth_past_year_per_1000,women_15_to_19_birth_past_year_per_1000,women_20_to_34_birth_past_year_per_1000,women_35_to_50_birth_past_year_per_1000,grdparents_live_w_grdchildren,grdparents_live_w_resp_grdchildren,resp_grdchildren_less_1yr,resp_grdchildren_1_or_2yrs,resp_grdchildren_3_or_4yrs,resp_grdchildren_5yrs_or_more,grdparents_resp_grdchildren,female_grdparents_resp_grdchildren,married_grdparents_resp_grdchildren,pop_3_and_over_in_school,nursery_and_preschool,kindergarten,elementary_school,high_school,college_or_graduate_school,pop_25_years_and_over,less_than_9th_grade,high_school_no_diploma,high_school_graduate,college_no_degree,associate_degree,bachelors_degree,graduate_or_prof_degree,pct_high_school_grad_or_higher,pct_bachelors_degree_or_higher,civilian_18_and_over,civilian_veterans,pop_civilian_noninst,pop_civilian_w_disability,age_under_18,age_under_18_w_disability,age_18_to_64,age_18_to_64_w_disability,age_65_and_over,age_65_and_over_w_disability,pop_1_and_over,res_same_house,res_diff_house_in_us,res_same_county,res_diff_county,res_diff_county_same_state,res_diff_county_and_state,res_abroad,total_pop_place_of_birth,pop_native,native_born_in_us,born_state_of_residence,born_different_state,born_islands_american_abroad,total_pop_foreign_born,pop_foreign_born,foreign_born_naturalized_us_citizen,foreign_born_not_us_citizen,pop_born_outside_us,native,native_entered_2010_or_later,native_entered_before_2010,foreign_born,foreign_born_entered_2010_or_later,foreign_born_entered_before_2010,foreign_born_excl_born_at_sea,foreign_born_europe,foreign_born_asia,foreign_born_africa,foreign_born_oceania,foreign_born_latin_america,foreign_born_northern_america,pop_5_and_over,english_language_only,non_english_language,non_english_poor_english,spanish_language,spanish_poor_english,other_indo_euro_language,other_indo_euro_poor_english,asian_language,asian_poor_english,other_language,english_less_than_v_well,total_pop_ancestory,pop_american,pop_arab,pop_czech,pop_danish,pop_dutch,pop_english,pop_french,pop_french_canadian,pop_german,pop_greek,pop_hungarian,pop_irish,pop_italian,pop_lithuanian,pop_norwegian,pop_polish,pop_portuguese,pop_russian,pop_scotch_irish,pop_scottish,pop_slovak,pop_subsaharan_africa,pop_swedish,pop_swiss,pop_ukrainian,pop_welsh,pop_west_indian
type,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int
mins,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [5]:
# Run GLRM to reduce ZCTA demographics to 10 archetypes
acs_model = H2OGeneralizedLowRankEstimator(k = 10,
                                           transform = "STANDARDIZE",
                                           loss = "Quadratic",
                                           regularization_x = "Quadratic",
                                           regularization_y = "L1",
                                           gamma_x = 0.25,
                                           gamma_y = 0.5,
                                           max_iterations = 100)
acs_model.train(x = acs_full.names, training_frame= acs_full)
print(acs_model)

glrm Model Build progress: |██████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLowRankEstimator :  Generalized Low Rank Modeling
Model Key:  GLRM_model_python_1614873327762_1

Model Summary: 


,number_of_iterations,final_step_size,final_objective_value
,100.0,0.0038692,1253523.7772765




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
RMSE: NaN
Sum of Squared Error (Numeric): 1250919.1282228571
Misclassification Error (Categorical): 0.0
Scoring History: 


,timestamp,duration,iterations,step_size,objective
,2021-03-04 09:56:00,13.160 sec,0.0,0.6666667,41041161.9728239
,2021-03-04 09:56:00,13.791 sec,1.0,0.4444444,41041161.9728239
,2021-03-04 09:56:01,14.501 sec,2.0,0.2222222,41041161.9728239
,2021-03-04 09:56:02,15.098 sec,3.0,0.0740741,41041161.9728239
,2021-03-04 09:56:02,15.721 sec,4.0,0.0185185,41041161.9728239
---,---,---,---,---,---
,2021-03-04 09:57:04,1 min 17.078 sec,95.0,0.0050136,1257282.2084656
,2021-03-04 09:57:04,1 min 17.699 sec,96.0,0.0052643,1255728.2694358
,2021-03-04 09:57:05,1 min 18.547 sec,97.0,0.0055275,1254373.4840273
,2021-03-04 09:57:06,1 min 19.142 sec,98.0,0.0058038,1253523.7772765



See the whole table with table.as_data_frame()



In [6]:
modelfile = acs_model.download_mojo(get_genmodel_jar=True)
print("Model saved to " + modelfile)

Model saved to /Users/RobertSmith/PycharmProjects/experimentation/GLRM_model_python_1614873327762_1.zip


In [7]:
archetypes = pd.DataFrame(acs_model.archetypes())

In [8]:
archetypes.to_csv("model_archetypes.csv", index = False)

In [9]:
archetypes

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
0,-1.810599,-1.662410,-1.370285,-1.745682,-1.288906,-1.138865,-1.330377,-1.222700,-1.249252,-1.868433,...,-0.726810,-2.090554,-2.210681,-0.840719,-0.418832,-1.538627,-1.235933,-0.836091,-1.886163,0.676641
1,1.917660,2.139134,2.979349,2.327245,3.758938,2.342065,1.791919,1.035182,0.777518,1.319066,...,-0.070118,0.121318,0.269836,-0.005256,1.336903,0.173444,0.026797,0.851208,0.147431,0.027119
2,4.940284,5.846776,6.884126,6.655110,8.862907,4.469796,2.114870,2.611800,1.963864,2.812762,...,6.766507,-0.404656,-0.228888,0.847378,0.103267,-0.012139,1.141578,3.460726,-0.128255,0.479292
3,1.302500,1.337881,1.341454,0.883874,0.777292,1.696239,1.615444,2.266732,2.275511,1.096511,...,-0.607551,0.805028,0.037891,-0.373774,1.287572,-0.761957,-0.513848,-0.401726,-0.230156,-0.266082
4,3.840906,3.625429,3.738207,2.295302,2.213049,4.635887,4.076883,6.426089,6.353531,3.791263,...,1.317526,0.581108,0.583387,0.308987,6.007328,-0.105635,-0.086839,1.152700,0.378359,8.768560
5,3.141620,3.151666,2.911284,3.693379,3.301527,1.649676,2.108309,1.351104,1.533399,2.776283,...,0.950896,4.123357,4.646496,1.932065,-0.132783,3.543990,3.032339,1.193955,4.292757,-1.348476
6,-0.575041,-0.526900,-0.441850,-0.523939,-0.374256,-0.413640,-0.513741,-0.456830,-0.475847,-0.595249,...,-0.226713,-0.634829,-0.783020,-0.197211,-0.323656,-0.638415,-0.464535,-0.264250,-0.664248,-0.024307
7,3.015123,3.684188,4.189117,3.174641,3.852376,5.179057,4.920582,3.870192,3.780197,1.514789,...,-0.512027,0.749235,0.654635,-0.560595,0.490676,0.437970,0.294109,-0.570083,0.450304,-1.155994
8,3.050043,2.631831,2.413111,3.039624,3.096344,1.866644,1.062604,1.094521,0.620049,3.441411,...,5.551989,1.088983,2.184103,0.556250,1.610037,1.988666,1.572275,4.342908,1.778300,0.224826
9,0.987345,0.981656,0.851830,1.329533,1.192233,0.244707,0.336432,-0.019726,-0.004102,0.887848,...,0.800027,1.176699,1.563932,1.056238,-0.501200,1.565891,1.280877,0.595961,1.438652,-0.317007
